In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from deeper.deeper_utils import wrap_deeper
from explanation.deepmatcher_utils import wrapDm
from explanation.mojito2 import explainSamples
import pandas as pd
import deepmatcher as dm
from keras.models import load_model
from deeper.DeepER import init_embeddings_index,init_embeddings_model

## Prelimary operations

In [ ]:
# Caricamento strutture dati e modelli ausiliari.
EMBEDDING_FILEPATH ='embeddings/glove.840B.300d.txt'
embeddings_index = init_embeddings_index(EMBEDDING_FILEPATH)
embeddings_model, tokenizer = init_embeddings_model(embeddings_index)

In [ ]:
def getCorrectPredictions(dataset,model,predict_fn):
    predictions = predict_fn(dataset,model,['label','id'])
    tp_group = dataset[(predictions[:,1]>=0.5)& (dataset['label'] == 1)]
    tn_group = dataset[(predictions[:,0] >=0.5)& (dataset['label']==0)]
    correctPredictions = pd.concat([tp_group,tn_group])
    print("Len of correct predictions is {}".format(len(correctPredictions)))
    return correctPredictions

In [ ]:
predict_fn_deeper = lambda data,model,ignore: wrap_deeper(data,'ltable_','rtable_',model,tokenizer,embeddings_model,
                                                  ignore_columns=ignore)

In [ ]:
predict_fn_deepmatcher = lambda data,model,ignore: wrapDm(data,model,ignore_columns=ignore)

## Walmart-Amazon

In [ ]:
augmented_test = pd.read_csv('datasets/walmart_amazon/augmented_test.csv')
walmart = pd.read_csv('datasets/walmart_amazon/walmart.csv',dtype=str)
amazon = pd.read_csv('datasets/walmart_amazon/amazon.csv',dtype=str)

In [ ]:
walmart_deepmatcher = dm.MatchingModel(attr_summarizer='rnn')
walmart_deepmatcher.load_state('models/walmart_amazon_rnn.pth')
walmart_deeper = load_model('models/DeepER_best_model_100_walmart-amazon.h5')

In [ ]:
correct_pred_deepmatcher = getCorrectPredictions(augmented_test,walmart_deepmatcher,predict_fn_deepmatcher)

In [ ]:
correct_pred_deeper = getCorrectPredictions(augmented_test,walmart_deeper,predict_fn_deeper)

In [ ]:
negrank,_,_ = explainSamples(correct_pred_deeper,[walmart,amazon],walmart_deeper,predict_fn_deeper,0,1)

In [ ]:
ax,negrank = aggregateRankings(negrank,positive=False,maxLenAttributes=1,lenTriangles=521)

In [ ]:
negrank,_,_ = explainSamples(correct_pred_deepmatcher,[walmart,amazon],walmart_deepmatcher,
                             predict_fn_deepmatcher,0,1)

In [ ]:
ax,negrank = aggregateRankings(negrank,positive=False,maxLenAttributes=1,lenTriangles=548)

In [ ]:
posrank_deeper,_,_ = explainSamples(correct_pred_deeper,[walmart,amazon],walmart_deeper,predict_fn_deeper,
                             1,1)

In [ ]:
posrank_deepmatcher,_,_ = explainSamples(correct_pred_deepmatcher,[walmart,amazon],walmart_deepmatcher,
                                         predict_fn_deepmatcher,1,1)

In [ ]:
ax,posrank = aggregateRankings(posrank_deeper,positive=True,maxLenAttributes=1,lenTriangles=521)

In [ ]:
ax,posrank = aggregateRankings(posrank_deepmatcher,positive=True,maxLenAttributes=1,lenTriangles=548)

## Amazon-Google

In [ ]:
augmented_test = pd.read_csv('datasets/Amazon-GoogleProducts/augmented_test.csv')
amazon = pd.read_csv('datasets/Amazon-GoogleProducts/Amazon.csv',dtype=str)
google = pd.read_csv('datasets/Amazon-GoogleProducts/Google.csv',dtype=str)

In [ ]:
amzgoogle_deepmatcher = dm.MatchingModel(attr_summarizer='rnn')
amzgoogle_deepmatcher.load_state('models/amazon_google_rnn.pth')
amzgoogle_deeper = load_model('models/DeepER_best_model_100_amazon-google.h5')

In [ ]:
correct_pred_deepmatcher = getCorrectPredictions(augmented_test,amzgoogle_deepmatcher,predict_fn_deepmatcher)
correct_pred_deeper = getCorrectPredictions(augmented_test,amzgoogle_deeper,predict_fn_deeper)

In [ ]:
negrank_deeper,_,_ = explainSamples(correct_pred_deeper,[amazon,google],amzgoogle_deeper,predict_fn_deeper,0,1)

In [ ]:
negrank_deepmatcher,_,_ = explainSamples(correct_pred_deepmatcher,[amazon,google],amzgoogle_deepmatcher
                                         ,predict_fn_deepmatcher,0,1)

In [ ]:
ax,negrank = aggregateRankings(negrank_deeper,positive=False,maxLenAttributes=1,lenTriangles=183)

In [ ]:
ax,negrank = aggregateRankings(negrank_deepmatcher,positive=False,maxLenAttributes=1,lenTriangles=303)

In [ ]:
posrank_deeper,_,_ = explainSamples(correct_pred_deeper,[amazon,google],amzgoogle_deeper,predict_fn_deeper,1,1)

In [ ]:
posrank_deepmatcher,_,_ = explainSamples(correct_pred_deepmatcher,[amazon,google],amzgoogle_deepmatcher,
                                         predict_fn_deepmatcher,1,1)

In [ ]:
ax,posrank = aggregateRankings(posrank_deeper,positive=True,maxLenAttributes=1,lenTriangles=183)

In [ ]:
ax,posrank = aggregateRankings(posrank_deepmatcher,positive=True,maxLenAttributes=1,lenTriangles=303)

## DBLP-ACM

In [ ]:
augmented_test = pd.read_csv('datasets/DBLP-ACM/augmented_test.csv')
dblp = pd.read_csv('datasets/DBLP-ACM/DBLP2.csv',dtype=str)
acm = pd.read_csv('datasets/DBLP-ACM/ACM.csv',dtype=str)

In [ ]:
dblpacm_deeper = load_model('models/DeepER_best_model_100_dblp-acm.h5')
dblpacm_deepmatcher = dm.MatchingModel(attr_summarizer='rnn')
dblpacm_deepmatcher.load_state('models/dblp_acm_rnn.pth')

In [ ]:
correct_deeper = getCorrectPredictions(augmented_test,dblpacm_deeper,predict_fn_deeper)
correct_deepmatcher = getCorrectPredictions(augmented_test,dblpacm_deepmatcher,predict_fn_deepmatcher)

In [ ]:
negrank_deeper,_,_ = explainSamples(correct_deeper,[dblp,acm],dblpacm_deeper,predict_fn_deeper,
                                   0,1)

In [ ]:
negrank_deepmatcher,_,_ = explainSamples(correct_deepmatcher,[dblp,acm],dblpacm_deepmatcher,predict_fn_deepmatcher,
                                        0,1)

In [ ]:
ax,negrank = aggregateRankings(negrank_deeper,positive=False,maxLenAttributes=1,lenTriangles=599)

In [ ]:
ax,negrank = aggregateRankings(negrank_deepmatcher,positive=False,maxLenAttributes=1,lenTriangles=629)

In [ ]:
posrank_deeper,_,_ = explainSamples(correct_deeper,[dblp,acm],dblpacm_deeper,predict_fn_deeper,
                                   1,1)

In [ ]:
posrank_deepmatcher,_,_ = explainSamples(correct_deepmatcher,[dblp,acm],dblpacm_deepmatcher,predict_fn_deepmatcher,
                                        1,1)

In [ ]:
ax,posrank = aggregateRankings(posrank_deeper,positive=True,maxLenAttributes=1,lenTriangles=599)

In [ ]:
ax,posrank = aggregateRankings(posrank_deepmatcher,positive=True,maxLenAttributes=1,lenTriangles=629)

## Fodors-Zagats

In [ ]:
augmented_test = pd.read_csv('datasets/fodors_zagats/augmented_test.csv')
fodors = pd.read_csv('datasets/fodors_zagats/fodors.csv',dtype=str)
zagats = pd.read_csv('datasets/fodors_zagats/zagats.csv',dtype=str)

In [ ]:
fodors_deeper = load_model('models/DeepER_best_model_100_fodors-zagats.h5')
fodors_deepmatcher = dm.MatchingModel(attr_summarizer='rnn')
fodors_deepmatcher.load_state('models/fodors_zagats_rnn.pth')

In [ ]:
correct_deeper = getCorrectPredictions(augmented_test,fodors_deeper,predict_fn_deeper)
correct_deepmatcher = getCorrectPredictions(augmented_test,fodors_deepmatcher,predict_fn_deepmatcher)

In [ ]:
negrank_deeper,_,_ = explainSamples(correct_deeper,[fodors,zagats],fodors_deeper,predict_fn_deeper,
                                   0,1)

In [ ]:
negrank_deepmatcher,_,_ = explainSamples(correct_deepmatcher,[fodors,zagats],fodors_deepmatcher,predict_fn_deepmatcher,
                                        0,1)

In [ ]:
ax,negrank = aggregateRankings(negrank_deepmatcher,positive=False,maxLenAttributes=1,lenTriangles=50)

In [ ]:
ax,negrank = aggregateRankings(negrank_deeper,positive=False,maxLenAttributes=1,lenTriangles=8)

In [ ]:
posrank_deepmatcher,_,_ = explainSamples(correct_deepmatcher,[fodors,zagats],fodors_deepmatcher,predict_fn_deepmatcher,
                                        1,1)

In [ ]:
posrank_deeper,_,_ = explainSamples(correct_deeper,[fodors,zagats],fodors_deeper,predict_fn_deeper,
                                   1,1)

In [ ]:
ax,posrank = aggregateRankings(posrank_deepmatcher,positive=True,maxLenAttributes=1,lenTriangles=50)

In [ ]:
ax,posrank = aggregateRankings(posrank_deeper,positive=True,maxLenAttributes=1,lenTriangles=8)

## Itunes-Amazon

In [ ]:
DATASET_DIR = 'datasets/itunes_amazon/'

In [ ]:
augmented_test = pd.read_csv(DATASET_DIR+'augmented_test.csv')
itunes = pd.read_csv(DATASET_DIR+'itunes.csv',dtype=str)
amazon = pd.read_csv(DATASET_DIR+'amazon.csv',dtype=str)

In [ ]:
itunes_deeper = load_model('models/DeepER_best_model_100_itunes-amazon.h5')
itunes_deepmatcher = dm.MatchingModel(attr_summarizer='rnn')
itunes_deepmatcher.load_state('models/itunes_amazon_rnn.pth')

In [ ]:
correct_deeper = getCorrectPredictions(augmented_test,itunes_deeper,predict_fn_deeper)
correct_deepmatcher = getCorrectPredictions(augmented_test,itunes_deepmatcher,predict_fn_deepmatcher)

In [ ]:
negrank_deeper,_,_ = explainSamples(correct_deeper,[itunes,amazon],itunes_deeper,predict_fn_deeper,
                                   0,1)

In [ ]:
negrank_deepematcher,_,_ = explainSamples(correct_deepmatcher,[itunes,amazon],itunes_deepmatcher,
                                         predict_fn_deepmatcher,0,1)

In [ ]:
posrank_deeper,_,_ = explainSamples(correct_deeper,[itunes,amazon],itunes_deeper,predict_fn_deeper,
                                   1,1)

In [ ]:
posrank_deepmatcher,_,_ = explainSamples(correct_deepmatcher,[itunes,amazon],itunes_deepmatcher,predict_fn_deepmatcher,
                                        1,1)

In [ ]:
ax,negrank = aggregateRankings(negrank_deeper,positive=False,maxLenAttributes=1,lenTriangles=66)

In [ ]:
ax,negrank = aggregateRankings(negrank_deepematcher,positive=False,maxLenAttributes=1,lenTriangles=76)

In [ ]:
ax,posrank = aggregateRankings(posrank_deeper,positive=True,ma)